# Update DB for recent data

In [18]:
import sqlite3
import pandas as pd

%cd /content/importforecasting_group2

# Connect to DB
conn = sqlite3.connect('economic_data.db')
cursor = conn.cursor()

try:
    cursor.execute('SELECT id, name, desc FROM series')
    series_data = cursor.fetchall()

    # Create a mapping of series id to (name, desc)
    series_map = {row[0]: (row[1], row[2]) for row in series_data}

    # Fetch from DB
    cursor.execute('''
        SELECT ei.date, ei.series, ei.value
        FROM economic_indicators ei
        JOIN series s ON ei.series = s.id
        ORDER BY ei.date, ei.series;
    ''')

    economic_indicators = cursor.fetchall()


    data = []

    for date, series_id, value in economic_indicators:
        name, desc = series_map[series_id]
        data.append({'date': date, 'series': name, 'value': value})

    econ_indicators_long = pd.DataFrame(data)

    # Pivot DF
    econ_indicators = econ_indicators_long.pivot(index='date', columns='series', values='value')

    # Convert the index to datetime
    econ_indicators.index = pd.to_datetime(econ_indicators.index)
    econ_indicators = econ_indicators.asfreq(freq='QS')


    latest_date = econ_indicators.index.max()

    # Step 7: Close the connection
    conn.close()

except sqlite3.Error as e:
    print(f"Error fetching data: {e}")
    # Step 7: Close the connection

finally:
    conn.close()


/content/importforecasting_group2


In [19]:
# Retrieve BEA Data using BEA API
  # GDP (2017 chained dollars)
  # Government Revenue from Customs Duties (2017 chained dollars)
  # Foreign transactions: total imports (2017 chained dollars)

import requests
import pandas as pd
from google.colab import userdata
from datetime import datetime


# Retrieve BEA API key from Google Colab secrets
BEA_API_key = userdata.get('BEA_API_key')

# ====== USER INPUT ======
user_id = BEA_API_key  # Replace with your actual BEA API key
tables = {
    "T10101": {"name": "gdp", "lines": ["1"]},  # Table 1.1.3 - GDP
    "T30200": {"name": "customs_receipts", "lines": ["6"]},  # Table 3.2 - Total Receipts
#    "T40206B": {"name": "Imports", "lines": ["94"]}  # Table 4.2.6B - Foreign Transactions: Imports
}
start_year = latest_date.year
end_year = datetime.today().year
frequency = "Q"  # A = Annual, Q = Quarterly, M = Monthly
# ========================

# Convert year range to comma-separated string
years = ",".join(str(year) for year in range(start_year, end_year + 1))

# Base API URL
base_url = "https://apps.bea.gov/api/data"

def fetch_bea_data(table_name, lines):
    """Fetches BEA data for a specific table and returns a formatted DataFrame."""
    params = {
        "UserID": user_id,
        "method": "GetData",
        "DataSetName": "NIPA",
        "TableName": table_name,
        "Frequency": frequency,
        "Year": years,
        "ResultFormat": "JSON"
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        try:
            df = pd.DataFrame(data["BEAAPI"]["Results"]["Data"])

            # Convert TimePeriod to datetime
            df["Date"] = pd.PeriodIndex(df["TimePeriod"], freq='Q').to_timestamp()

            # Ensure numerical values
            df['DataValue'] = pd.to_numeric(df['DataValue'].str.replace(',', '', regex=False), errors='coerce')

            # Filter only the required lines
            df = df[df["LineNumber"].isin(lines)]

            # Debugging: Print missing values in time periods
            print(f"\nFetched {table_name}: {df['TimePeriod'].nunique()} unique periods")

            # Pivot the table safely
            df = df.pivot_table(index="Date", columns="LineDescription", values="DataValue", aggfunc="first")

            # Rename columns with table identifiers
            df.columns = [f"{col} ({tables[table_name]['name']})" for col in df.columns]

            return df
        except KeyError:
            print(f"Unexpected JSON structure for Table {table_name}: {data}")
            return None
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Fetch data for selected tables & lines
dataframes = {table: fetch_bea_data(table, info["lines"]) for table, info in tables.items()}
dataframes = {k: v for k, v in dataframes.items() if v is not None}  # Remove None values

# Standardize the date index before merging
# for key, df in dataframes.items():
#    df.index = df.index.to_period("Q")  # Convert to quarterly periods to align

# Merge all DataFrames on TimePeriod
if dataframes:
    # Start merging with the first DataFrame
    merged_df = None
    for key, df in dataframes.items():
        if merged_df is None:
            merged_df = df
        else:
            merged_df = merged_df.merge(df, left_index=True, right_index=True, how="outer")  # Keep all data

    # Debugging: Print rows with NaN values
    missing_values = merged_df.isna().sum(axis=1)
    if missing_values.any():
        print("\nRows with missing values (check for time misalignment):")
        print(merged_df[missing_values > 0])

    # Display the merged DataFrame
    print("\nMerged DataFrame:")
    print(merged_df.head())
    print(merged_df.dtypes)
else:
    print("No data retrieved from BEA API.")


Fetched T10101: 4 unique periods

Fetched T30200: 4 unique periods

Merged DataFrame:
            Gross domestic product (gdp)  Customs duties (customs_receipts)
Date                                                                       
2024-01-01                           1.6                              82216
2024-04-01                           3.0                              78075
2024-07-01                           3.1                              85126
2024-10-01                           2.4                              86460
Gross domestic product (gdp)         float64
Customs duties (customs_receipts)      int64
dtype: object


In [20]:
import requests
import pandas as pd
from google.colab import userdata
from datetime import datetime

# Retrieve FRED API key from Google Colab secrets
FRED_API_key = userdata.get('FRED_API_key')

# ====== USER INPUT ======
series_ids = {
    "Federal Funds Rate (fedfunds)": "FEDFUNDS",
    "Real Imports (imports)": "IMPGSC1"
}
start_date = datetime.strftime(latest_date, "%Y-%m-%d")
end_date = datetime.today().strftime("%Y-%m-%d")
frequency = "q"  # Quarterly
# ========================

# Base API URL for FRED
fred_base_url = "https://api.stlouisfed.org/fred/series/observations"

# Function to fetch a FRED series into a DataFrame
def fetch_fred_series(series_id, series_name):
    params = {
        "series_id": series_id,
        "api_key": FRED_API_key,
        "file_type": "json",
        "observation_start": start_date,
        "observation_end": end_date,
        "frequency": frequency,
        "aggregation_method": "avg"
    }

    response = requests.get(fred_base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data["observations"])
        df["date"] = pd.to_datetime(df["date"])
        df.set_index("date", inplace=True)
        df = df.drop(columns=["realtime_start", "realtime_end"])
        df["value"] = pd.to_numeric(df["value"], errors="coerce")
        df.rename(columns={"value": series_name}, inplace=True)
        return df
    else:
        print(f"Error fetching {series_id}: {response.status_code}")
        return pd.DataFrame()

# Fetch both series
fedfunds_df = fetch_fred_series("FEDFUNDS", "Federal Funds Rate (fedfunds)")
imports_df = fetch_fred_series("IMPGSC1", "Real Imports (imports)")

# Merge with existing BEA dataset
final_merged_df = merged_df.join(fedfunds_df, how="left")
final_merged_df = final_merged_df.join(imports_df, how="left")

# Display result
print(final_merged_df.head())
print(final_merged_df.dtypes)


            Gross domestic product (gdp)  Customs duties (customs_receipts)  \
Date                                                                          
2024-01-01                           1.6                              82216   
2024-04-01                           3.0                              78075   
2024-07-01                           3.1                              85126   
2024-10-01                           2.4                              86460   

            Federal Funds Rate (fedfunds)  Real Imports (imports)  
Date                                                               
2024-01-01                            NaN                     NaN  
2024-04-01                            NaN                     NaN  
2024-07-01                            NaN                     NaN  
2024-10-01                           4.65                3689.829  
Gross domestic product (gdp)         float64
Customs duties (customs_receipts)      int64
Federal Funds Rate (fedfund

In [21]:
# IF NECESSARY: clear series and ei tables
'''
import sqlite3

conn = sqlite3.connect('economic_data.db')
cursor = conn.cursor()

# Clear the tables
cursor.execute("DELETE FROM series")
cursor.execute("DELETE FROM economic_indicators")

conn.commit()
conn.close()
'''

'\nimport sqlite3\n\nconn = sqlite3.connect(\'economic_data.db\')\ncursor = conn.cursor()\n\n# Clear the tables\ncursor.execute("DELETE FROM series")\ncursor.execute("DELETE FROM economic_indicators")\n\nconn.commit()\nconn.close()\n'

In [22]:
import sqlite3
import pandas as pd
import re

# Insert data into the 'series' table
# Loop through the columns of the DataFrame and extract 'name' and 'desc' from column names

conn = sqlite3.connect('economic_data.db')
cursor = conn.cursor()

try:

    existing_series = set()

    for col in final_merged_df.columns:
        # Extract 'desc' (before parentheses) and 'name' (inside parentheses)
        match = re.match(r"(.*)\s\((.*)\)", col)
        if match:
            desc = match.group(1)  # The part before the parentheses
            name = match.group(2)  # The part inside the parentheses

            # Skip if this series already exists
            if (name, desc) not in existing_series:
                cursor.execute('''
                    INSERT OR IGNORE INTO series (name, desc)
                    VALUES (?, ?);
                ''', (name, desc))
                existing_series.add((name, desc))

    # Commit the changes to the 'series' table
    conn.commit()

    # Insert data into the 'economic_indicators' table
    # For each column in the DataFrame, fetch the corresponding series id and insert the data

    for col in final_merged_df.columns:
        # Extract 'desc' (before parentheses) and 'name' (inside parentheses)
        match = re.match(r"(.*)\s\((.*)\)", col)
        if match:
            desc = match.group(1)  # The part before the parentheses
            name = match.group(2)  # The part inside the parentheses

            # Fetch the series ID from the 'series' table
            cursor.execute('''
                SELECT id FROM series WHERE name = ? AND desc = ?;
            ''', (name, desc))
            series_id = cursor.fetchone()[0]  # Get the series ID

    for idx, row in final_merged_df.iterrows():
        # Check if the date already exists in the database for this series
        cursor.execute('''
            SELECT 1 FROM economic_indicators WHERE date = ? AND series = ? LIMIT 1;
        ''', (str(idx), series_id))

        # If no rows are returned, insert the new data
        if cursor.fetchone() is None:
            cursor.execute('''
                INSERT INTO economic_indicators (date, series, value)
                VALUES (?, ?, ?);
            ''', (str(idx), series_id, row[col]))

# Commit the transaction to save the changes
    conn.commit()

    # Commit the changes to the 'economic_indicators' table

except sqlite3.Error as e:
    print(f"Error inserting data: {e}")

finally:
    # Close the connection
    conn.close()

In [24]:
import sqlite3
import pandas as pd

%cd /content/importforecasting_group2

# Connect to DB
conn = sqlite3.connect('economic_data.db')
cursor = conn.cursor()

try:
    cursor.execute('SELECT id, name, desc FROM series')
    series_data = cursor.fetchall()

    # Create a mapping of series id to (name, desc)
    series_map = {row[0]: (row[1], row[2]) for row in series_data}

    # Fetch from DB
    cursor.execute('''
        SELECT ei.date, ei.series, ei.value
        FROM economic_indicators ei
        JOIN series s ON ei.series = s.id
        ORDER BY ei.date, ei.series;
    ''')

    economic_indicators = cursor.fetchall()


    data = []

    for date, series_id, value in economic_indicators:
        name, desc = series_map[series_id]
        data.append({'date': date, 'series': name, 'value': value})

    econ_indicators_long = pd.DataFrame(data)

    # Pivot DF
    econ_indicators = econ_indicators_long.pivot(index='date', columns='series', values='value')

    # Convert the index to datetime
    econ_indicators.index = pd.to_datetime(econ_indicators.index)
    econ_indicators = econ_indicators.asfreq(freq='QS')



    # Step 7: Close the connection

except sqlite3.Error as e:
    print(f"Error fetching data: {e}")
    # Step 7: Close the connection

finally:
    conn.close()

econ_indicators.tail()

/content/importforecasting_group2


series,customs_receipts,fedfunds,gdp,imports
date,,,,
2023-10-01,79531.0,5.33,3.2,3496.290
2024-01-01,82216.0,5.33,1.6,3548.749
2024-04-01,78075.0,5.33,3.0,3614.047
2024-07-01,85126.0,5.26,3.1,3707.429
2024-10-01,86460.0,4.65,2.4,3689.829
